### LexRank algoritam

`LexRank` algoritam je algoritam za ekstraktivnu sumarizaciju teksta koji koristi ideju opisanog `PageRank` algoritma. Zadatak esktraktivne sumarizacije je da za zadati tekst generiše kraću verziju izdvajajući samo rečenice koje su najinformativnije.  <img src='assets/summarization.jpg'>

Čvorovi grafa će sada biti rečenice $s_i$, ivice će povezivati rečenice koje dele informacioni sadržaj, a njihove težine $w_{ij}$ će odgovarati količini preklapanja informacionih sadržaja.  <img src='assets/lexrank_color.png' style='width:400px'>

#### Obrada teksta

Da bismo pripremili tekst za rad, iskoristićemo `nltk` (skraćenica od *natural language toolkit*) paket koji se koristi za rad sa tekstom, a koji nudi podršku za zadatke tokenizacije, tagiranja, normalizacije i slično. Alat se može instalirati komandom `conda install -c anaconda nltk` u skladu sa smernicama sa [zvaničnog sajta](https://anaconda.org/anaconda/nltk). 

Punkt je ime podalata koji se koristi za razbija tekst na rečenice i tokene. 

In [ ]:
import nltk
nltk.download('punkt')

U radu ćemo koristiti članke o tenisu dostupne u `tennis_articles.csv` datoteci. 

In [ ]:
import pandas as pd

Koraci koje ćemo preduzeti za svaki članak prikazani su na slici. <img src='assets/summarization_pipeline.png' style='width:500px'> Mi ćemo raditi nad pojedinačnim člancima pa je potrebno preskočiti korak *combine*.

Pojedinačne tekstove ćemo razbiti na rečenice korišćenjem `nltk` paketa i njegove funkcije `sent_tokenize`. 

Dalje ćemo na nivou rečenica izdvojiti pojedinačne reči. Prilikom izdvajanja reči eliminisaćemo interpunkciju i takozvane stop reči koje se jako često pojavljuju i koje ne doprinose svojim značenjem (npr. reči poput članova ili predloga).

Za izdvajanje reči koristićemo funkciju `word_tokenize`. Ova funkcija očekuje segment teksta, a kao rezultat vraća listu reči.

Stop reči možemo pročitati iz paketa `stopwords` za svaki od jezika koji podržava biblioteka `nltk`.

Interpunkcijske karaktere ćemo očitati iz `string` paketa.

Na gore opisane načine, sada ćemo pripremiti izdvojene rečenice.

Da bismo dobili vektorsku reprezentaciju teksta, iskoristićemo GloVe pritrenirane distribuirane reprezentacije reči. Alternativa su mogli biti i n-gramski profili rečenica ili njihove Tf-Idf reprezentacije, ali se u praksi dobijaju bolji rezultati ukoliko se iskoriste ovakve reprezentacije. Ceo paket sa reprezentacijama reči različitih dužina (50, 100, 200 i 300) ukupne veličine 822MB se može preuzeti sa [zvanične adrese](https://nlp.stanford.edu/projects/glove/). Mi ćemo u radu koristiti reprezentacije dužine 100 koje se nalaze u datoteci `glove.6B.100d.txt` (347.1MB) koja se može preuzeti pojedinačno npr. sa [ove](https://www.kaggle.com/terenceliu4444/glove6b100dtxt) adrese. 

Prvo ćemo pročitati iz preuzete datoteke sve podržane reči i njihove vektorske reprezentacije. U pojedinačnim redovima datoteke se prvo nalazi reči, a potom 100 realnih vrednosti koje predstavljaju njenu vektorsku reprezentaciju. 

In [ ]:
import numpy as np

#### Kreiranje grafa

Dalje je potrebno pripremiti matricu grafa čiji elementi predstavljaju meru sličnosti između rečenica. Za računanje sličnosti između rečenica koristićemo kosinusnu sličnost. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

Za rangiranje rečenica u sažetku iskoristićemo bibliotečku `pagerank` funkciju. 

In [ ]:
import networkx as nx